In [ ]:
import librosa # Lo usaremos para el analisis y procesamiento de audio
from IPython.display import Audio # para poder reproducir audio dentro del notebook
import matplotlib as plt 
import numpy as np
from scipy.io import wavfile # para escribir o leer audios
import warnings 
import py7zr # para comprimir descomprimir archivos